# Generation of files for media preparation using biomek

This notebook...

## Inputs and outputs

#### Required files to run this notebook:
   - `../data/standard_recipe_concentrations.csv` - this file will not change over the course of a particular project
   
   - `../data/stock_concentrations.csv` - this file will not change over the course of a particular project
   
   - `../data/DBTL0/target_concentrations.csv` - this is an output from an ART run and it will change at every DBTL cycle

   

#### Files generated by running this notebook:


   - `stock_plate.csv` - instructions on how to prepare the source plate
   
   
The files that need to be uploaded to biomek, following this particular order:
   
   - `P200_water.csv`
   
   - `P20_water.csv`
         
   - `P20_kan.csv` 
   
   - `P200_components.csv` 
   
   - `P20_components.csv` 
  
   - `P20_culture.csv` 
   
   
Note that all these different files are needed as biomek needs a separate one for different types of source/destination plate, tips. Also it cannot follow the order of operations within a file so we have to provide the order using different files (e.g. first a file with kan transfers, then a file with components transfers).
    
The files are stored in the user defined directory. 

## Setup

Importing needed libraries:

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np

from comb_media.core import find_volumes

/Users/tradivojevic/.opentrons/deck_calibration.json not found. Loading defaults
/Users/tradivojevic/.opentrons/robot_settings.json not found. Loading defaults


Loading json containers...
Json container file load complete, listing database
Found 0 containers to add. Starting migration...
Database migration complete!


## User parameters

In [2]:
user_params = {
    'stock_conc_file': '../data/stock_concentrations_500uL.csv',
    'standard_media_file': '../data/standard_recipe_concentrations.csv',
    'target_conc_file': '../data/DBTL0/96well_dest_plate/target_concentrations.csv',
    'output_path': '../data/DBTL0/96well_dest_plate',  # Path for output files
    'well_volume': 500,            # Total volume of the media content in the destination well
    'tips': ['f20', 'f200'],        # Choose available tips from f20, s20, f50, s50, f200, s200
    'min_transfer_volume': 5.,      # Minimal transfer volume of the liquid handler
    'culture_factor': 100           # Dilution factor for culture, e.g. 100x, 1000x

}

Setup tips

In [3]:
df_tips = pd.DataFrame([['f20', 36],
                        ['s20', 72],
                        ['f50', 45],
                        ['s50', 81],
                        ['f200', 135],
                        ['s200', 171]],
                       columns=['Tips', 'Max Volume'],
                       ).set_index('Tips')


In [4]:
max_tip_volume = max([df_tips.loc[tip]['Max Volume'] for tip in user_params['tips']])
max_tip_volume

135

Load the standard media recipe

In [5]:
df_stand = pd.read_csv(user_params['standard_media_file']).set_index("Component")

Load the stock concentrations

In [6]:
df_stock = pd.read_csv(user_params['stock_conc_file'])
df_stock = df_stock.set_index("Component")

In [7]:
df_stock

,Low Concentration[mM],High Concentration[mM],Dilution Factor
Component,,,
MOPS,2000.0000,2000.0000,1.0
Tricine,400.0000,400.0000,1.0
H3BO3,0.0400,0.8000,20.0
Glucose,2000.0000,2000.0000,1.0
K2SO4,2.9000,78.3000,27.0
K2HPO4,26.4000,132.0000,5.0
FeSO4,0.1000,2.0000,20.0
NH4Cl,634.6667,634.6667,1.0
MgCl2,5.2000,46.8000,9.0


## Create stock solutions plate dataframe


In [8]:
df_stock_plate = pd.DataFrame(
    columns=["Component", "Stock", "Concentration[mM]"])
    
i = 0
num_comp = len(df_stock)
for i in range(2*num_comp):

    if i < num_comp:
        component = df_stock.index[i]
        df_stock_plate.loc[i] = [
            component, 
            "high",
            df_stock.loc[component]["High Concentration[mM]"]
        ]
    elif i < 2*num_comp:
        component = df_stock.index[(i-num_comp)]
        if df_stock.loc[component]["Dilution Factor"] > 1.0:
            df_stock_plate.loc[i] = [
                component, 
                "low",
                df_stock.loc[component]["Low Concentration[mM]"]
            ]
            
df_stock_plate.reset_index(drop=True, inplace=True)

In [9]:
df_stock_plate

,Component,Stock,Concentration[mM]
0,MOPS,high,2000.0000
1,Tricine,high,400.0000
2,H3BO3,high,0.8000
3,Glucose,high,2000.0000
4,K2SO4,high,78.3000
5,K2HPO4,high,132.0000
6,FeSO4,high,2.0000
7,NH4Cl,high,634.6667
8,MgCl2,high,46.8000
9,NaCl,high,1500.0000


We will not save this stock plate for now, as likely not all wells/concentration levels will be needed once we load the target concentrations.

## Read target concentrations and convert to volumes

Read ART suggested target concentrations. Note that those are only for components which are being explored. 

In [11]:
target_conc_file = user_params['target_conc_file']
df_target_conc = pd.read_csv(target_conc_file, index_col=0)
df_target_conc.head()

,H3BO3,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4
A1,0.004,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001
B1,0.004,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001
C1,0.004,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001
D1,0.004,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001
E1,0.004,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001


### Add fixed components and antibiotic concentrations to the target concentrations dataframe

Find fixed components as those from the standard recipe that are not listed in target concentration file, which is an output from ART

In [12]:
comp_fixed = list(df_stand.drop(df_target_conc.columns).index)
print('Fixed components: ')
for comp in comp_fixed:
    df_target_conc[comp] = df_stand.at[comp, 'Concentration[mM]']
    print(f'{comp}')

Fixed components: 
MOPS
Tricine
Glucose
Kan


Make sure the order of columns is the same as in the stock dataframe:

In [13]:
columns = df_stock.index
df_target_conc = df_target_conc[columns]

In [14]:
df_target_conc.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan
A1,40.0,4.0,0.004,20.0,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001,0.08582
B1,40.0,4.0,0.004,20.0,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001,0.08582
C1,40.0,4.0,0.004,20.0,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001,0.08582
D1,40.0,4.0,0.004,20.0,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001,0.08582
E1,40.0,4.0,0.004,20.0,0.29,1.32,0.01,9.52,0.52,50,0.00003,0.0003,0.0001,0.0008,0.0001,0.08582


## Calculate all transfer volumes

Also, create a dataframe with levels of stock concentrations needed to achieve those volumes, which will indicate from which source well the transfer should be made

In [15]:
verbose = 0
EPS = 0.000001
min_tip_volume = user_params['min_transfer_volume']

df_conc_level = pd.DataFrame(data='high',
    index=df_target_conc.index,
    columns=df_target_conc.columns)

df_volumes = df_target_conc.copy()

# Add column for water
df_volumes['Water'] = None

for i in range(len(df_volumes)):
    volumes, df = find_volumes(
        user_params['well_volume'], 
        components=df_stock.index,
        stock_conc_val=df_stock["High Concentration[mM]"].values, 
        target_conc_val=df_target_conc.iloc[i].values,
        culture_ratio=user_params['culture_factor']
    )
    df_volumes.iloc[i] = volumes
    
    # Find volumes smaller than min transfer volume and use low concentrations for those components
    comp_small_vol = df_volumes.iloc[i][
        df_volumes.iloc[i] < min_tip_volume - EPS
    ].index
    if verbose:
        print(f'Compoments small: {comp_small_vol}')
    
    stock_new = df_stock["High Concentration[mM]"].copy()
    for comp in comp_small_vol:
        stock_new[comp] = df_stock.loc[comp]["Low Concentration[mM]"]
        df_conc_level.iloc[i][comp] = 'low'

# Recalculate  volumes
for i in range(len(df_volumes)):
    volumes, df = find_volumes(
        user_params['well_volume'], 
        components=df_stock.index,
        stock_conc_val=stock_new.values, 
        target_conc_val=df_target_conc.iloc[i].values,
        culture_ratio=user_params['culture_factor']
    )
    df_volumes.iloc[i] = volumes
    
display(df_volumes.head(5))


,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan,Water
A1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778
B1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778
C1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778
D1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778
E1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778


In [16]:
df_conc_level.head(5)

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan
A1,high,high,low,high,low,high,low,high,high,high,low,low,low,low,low,high
B1,high,high,low,high,low,high,low,high,high,high,low,low,low,low,low,high
C1,high,high,low,high,low,high,low,high,high,high,low,low,low,low,low,high
D1,high,high,low,high,low,high,low,high,high,high,low,low,low,low,low,high
E1,high,high,low,high,low,high,low,high,high,high,low,low,low,low,low,high


Add volumes for culture

In [17]:
df_volumes['Culture'] = user_params['well_volume'] / user_params['culture_factor']
df_volumes.head()

,MOPS,Tricine,H3BO3,Glucose,K2SO4,K2HPO4,FeSO4,NH4Cl,MgCl2,NaCl,(NH4)6Mo7O24,CoCl2,CuSO4,MnSO4,ZnSO4,Kan,Water,Culture
A1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778,5.0
B1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778,5.0
C1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778,5.0
D1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778,5.0
E1,10.0,5.0,50.0,5.0,50.0,5.0,50.0,7.5,5.555556,16.666667,50.0,50.0,50.0,50.0,50.0,5.0,35.2778,5.0


Check if sum of all volumes is equal to total well volume

In [18]:
assert (np.sum(df_volumes.values, axis=1) == user_params['well_volume']).all(), 'Sum of all volumes is not equal to total well volume!' 

### Save volumes in the destination wells to a file

In [19]:
volumes_file = f'{user_params["output_path"]}/dest_volumes_96well.csv'
df_volumes.to_csv(volumes_file)

### Calculate total volumes needed in the source wells

In [20]:
tot_vol_water = np.sum(df_volumes['Water'].values)
print(f'Total volume of water needed: {tot_vol_water:.0f} uL + dead volume')

Total volume of water needed: 1693 uL + dead volume


In [21]:
for i in range(len(df_stock_plate)):
    comp = df_stock_plate.loc[i, 'Component']
    stock_level = df_stock_plate.loc[i, 'Stock']
    tot_vol_comp = np.sum(
        df_volumes[df_conc_level[comp]==stock_level][comp].values
    )
    df_stock_plate.loc[i, 'Volume [uL]'] = np.round(tot_vol_comp)

# Add culture to the stock plate
tot_vol_culture = np.sum(df_volumes['Culture'].values) 
culture = {'Component': 'Culture', 
           'Stock': '', 
           'Concentration[mM]': None,
           'Volume [uL]': np.round(tot_vol_culture)
          }
df_stock_plate = df_stock_plate.append(culture, ignore_index = True)


As not both levels for all components are needed for this run, let's remove them from the source plate.

In [22]:
df_stock_plate = df_stock_plate[df_stock_plate['Volume [uL]'] > 0]
df_stock_plate.reset_index(drop=True, inplace=True)

In [23]:
df_stock_plate

,Component,Stock,Concentration[mM],Volume [uL]
0,MOPS,high,2000.0000,480.0
1,Tricine,high,400.0000,240.0
2,Glucose,high,2000.0000,240.0
3,K2HPO4,high,132.0000,240.0
4,NH4Cl,high,634.6667,360.0
5,MgCl2,high,46.8000,267.0
6,NaCl,high,1500.0000,800.0
7,Kan,high,8.5820,240.0
8,H3BO3,low,0.0400,2400.0
9,K2SO4,low,2.9000,2400.0


Define the source plate:

In [24]:
num_source_wells = len(df_stock_plate)

if num_source_wells <= 24:
    source_well_type = '24-well'
    well_volume = 9000  # including dead volume
    dead_volume = 50
else:
    source_well_type = '48-well'
    well_volume = 4000  # including dead volume
    dead_volume = 30


Are the total volumes smaller than well volume of the plate?

In [25]:
df_new_wells = pd.DataFrame(columns=df_stock_plate.columns)
ind_drop = []
for i in range(len(df_stock_plate)):
    tot_volume = df_stock_plate.iloc[i]['Volume [uL]']
    if tot_volume + dead_volume > well_volume:
        ind_drop = ind_drop + [i]
        comp = df_stock_plate.iloc[i]['Component']
        conc_level = df_stock_plate.iloc[i]['Stock']
        num_wells_needed = np.ceil(tot_volume / well_volume)
        print(f'We need {num_wells_needed:.0f} well(s) for {comp} with {conc_level} concentration')
        # Include additional wells needed
        indices = np.linspace(i, i+1, num=num_wells_needed, endpoint=False)
        volumes = []
        volume_left = tot_volume + num_wells_needed*dead_volume
        while volume_left > dead_volume:
            volumes = volumes + [min(volume_left, well_volume)] 
            volume_left = volume_left - volumes[-1]
            
        d = {'Component' : pd.Series(comp, index=indices),
             'Stock' : pd.Series(conc_level, index=indices),
             'Concentration[mM]': pd.Series(df_stock_plate.iloc[0]['Concentration[mM]'], index =indices),
             'Volume [uL]': pd.Series(volumes, index=indices)}

        df = pd.DataFrame(d)
        df_new_wells = df_new_wells.append(df, ignore_index=False)        

    else:
        df_stock_plate.at[i,'Volume [uL]'] += dead_volume
        print('.', end='')
print('Finished.')

df_stock_plate.drop(ind_drop, inplace=True)
df_stock_plate = df_stock_plate.append(df_new_wells, ignore_index=False)
df_stock_plate = df_stock_plate.sort_index().reset_index(drop=True) 


.................Finished.


Assign well names:

In [26]:
num_source_wells = len(df_stock_plate) 

if num_source_wells <= 24:
    source_well_type = '24-well'
    print(f'Use {source_well_type} source plate')
    well_rows = 'ABCD'
    well_columns = '123456'
else:
    source_well_type = '48-well'
    print(f'Use {source_well_type} source plate')
    well_rows = 'ABCDEF'
    well_columns = '12345678'

Use 24-well source plate


In [27]:
well_names = [f'{row}{column}' for column in well_columns for row in well_rows]
well_names = well_names[:num_source_wells]

df_stock_plate.reset_index(drop=True, inplace=True)
df_stock_plate['Well'] = well_names
df_stock_plate = df_stock_plate.set_index(['Well'])

Final stock plate:

In [28]:
df_stock_plate

,Component,Stock,Concentration[mM],Volume [uL]
Well,,,,
A1,MOPS,high,2000.0000,530.0
B1,Tricine,high,400.0000,290.0
C1,Glucose,high,2000.0000,290.0
D1,K2HPO4,high,132.0000,290.0
A2,NH4Cl,high,634.6667,410.0
B2,MgCl2,high,46.8000,317.0
C2,NaCl,high,1500.0000,850.0
D2,Kan,high,8.5820,290.0
A3,H3BO3,low,0.0400,2450.0


In [29]:
stock_plate_file = f"{user_params['output_path']}/{source_well_type}_stock_plate.csv"


In [30]:
df_stock_plate.to_csv(stock_plate_file)

## Define transfers for biomek

In [31]:
column_names = [
    "Source Position",
    "Source Well",
    "Destination Position",
    "Destination Well",
    "Transfer Volume [uL]"
]

### Create water transfers

In [32]:
P20_water = pd.DataFrame(columns=column_names)
P200_water = P20_water.copy()

i = 0  # counter for index

comp = 'Water'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_water.at[i, "Destination Well"] = dest_well
        P20_water.at[i, "Transfer Volume [uL]"] = vol

    # P200 transfer
    else:
        P200_water.at[i, "Destination Well"] = dest_well
        if vol <= max_tip_volume:
            P200_water.at[i, "Transfer Volume [uL]"] = vol
        else:
            # Divide the transfer in parts until whole volume is transfered
            # First transfer
            transf_vol = max_tip_volume
            P200_water.at[i, "Transfer Volume [uL]"] = transf_vol
            volume_left = vol - transf_vol
            # Other transfers
            while volume_left > 0:
                transf_vol = min(max_tip_volume, volume_left)
                i += 1
                if transf_vol < 30:
                    P20_water.at[i, "Destination Well"] = dest_well
                    P20_water.at[i, "Transfer Volume [uL]"] = transf_vol
                else:
                    P200_water.at[i, "Destination Well"] = dest_well
                    P200_water.at[i, "Transfer Volume [uL]"] = transf_vol
                volume_left = volume_left - transf_vol

    i += 1

P20_water["Source Position"] = "P3" 
P20_water["Source Well"] = "A1"  # reservoir plate
P20_water["Destination Position"] = "P2" 
P200_water["Source Position"] = "P3" 
P200_water["Source Well"] = "A1"
P200_water["Destination Position"] = "P2" 

### Create kan transfers

In [33]:
P20_kan = pd.DataFrame(columns=column_names)
P200_kan = P20_kan.copy()

i = 0  # counter for index

comp = 'Kan'
    
for dest_well in df_volumes.index:
    vol = df_volumes.at[dest_well, comp]

    # P20 transfer
    if vol < 30:
        P20_kan.at[i, "Destination Well"] = dest_well
        P20_kan.at[i, "Transfer Volume [uL]"] = vol

    # P200 transfer
    else:
        P200_kan.at[i, "Destination Well"] = dest_well
        if vol <= max_tip_volume:
            P200_kan.at[i, "Transfer Volume [uL]"] = vol
        else:
            # Divide the transfer in parts until whole volume is transfered
            # First transfer
            transf_vol = max_tip_volume
            P200_kan.at[i, "Transfer Volume [uL]"] = transf_vol
            volume_left = vol - transf_vol
            # Other transfers
            while volume_left > 0:
                i += 1
                transf_vol = min(max_tip_volume, volume_left)
                if transf_vol < 30:
                    P20_kan.at[i, "Destination Well"] = dest_well
                    P20_kan.at[i, "Transfer Volume [uL]"] = transf_vol
                else:
                    P200_kan.at[i, "Destination Well"] = dest_well
                    P200_kan.at[i, "Transfer Volume [uL]"] = transf_vol
                volume_left = volume_left - transf_vol

    i += 1

P20_kan["Source Position"] = "P1" 
P20_kan["Source Well"] = df_stock_plate[
    df_stock_plate["Component"]==comp
].index[0]  
P20_kan["Destination Position"] = "P2"
P200_kan["Source Position"] = "P1" 
P200_kan["Source Well"] = df_stock_plate[
    df_stock_plate["Component"]==comp
].index[0]  
P200_kan["Destination Position"] = "P2"


### Create culture transfers

Note that we assume here the volumes for culture are small enough for p20 pipette, if that is not the case you need to adjust.

In [34]:
P20_culture = pd.DataFrame(columns=column_names)

comp = 'Culture'
vol = df_volumes[comp][0]
    
for i, dest_well in enumerate(df_volumes.index):
    
    P20_culture.at[i, "Destination Well"] = dest_well
    P20_culture.at[i, "Transfer Volume [uL]"] = vol

P20_culture["Source Position"] = "P1" 
P20_culture["Source Well"] = df_stock_plate[
    df_stock_plate["Component"]==comp
].index[0]  
P20_culture["Destination Position"] = "P2" 


## Create component transfers

In [35]:
P20_components = pd.DataFrame(columns=column_names) 
P200_components = P20_components.copy()

In [36]:
i = 0  # counter for index

components = list(df_volumes.columns.drop(['Kan', 'Water', 'Culture']))

for comp in components:
    
    for dest_well in df_volumes.index:
        vol = df_volumes.at[dest_well, comp]
        conc_level = df_conc_level.at[dest_well, comp]
        source_well = df_stock_plate[
            (df_stock_plate["Component"]==comp) &
            (df_stock_plate["Stock"]==conc_level)
        ].index[0]
        
        # P20 transfer
        if vol < 30:
            P20_components.at[i, "Source Well"] = source_well
            P20_components.at[i, "Destination Well"] = dest_well
            P20_components.at[i, "Transfer Volume [uL]"] = vol

        # P200 transfer
        else:
            P200_components.at[i, "Source Well"] = source_well
            P200_components.at[i, "Destination Well"] = dest_well
            if vol <= max_tip_volume:
                P200_components.at[i, "Transfer Volume [uL]"] = vol
            else:
                # Divide the transfer in parts until whole volume is transfered
                # First transfer
                transf_vol = max_tip_volume
                P200_components.at[i, "Transfer Volume [uL]"] = transf_vol
                volume_left = vol - transf_vol
                # Other transfers
                while volume_left > 0:
                    i += 1
                    transf_vol = min(max_tip_volume, volume_left)
                    if transf_vol < 30:
                        P20_components.at[i, "Source Well"] = source_well
                        P20_components.at[i, "Destination Well"] = dest_well
                        P20_components.at[i, "Transfer Volume [uL]"] = transf_vol
                    else:
                        P200_components.at[i, "Source Well"] = source_well
                        P200_components.at[i, "Destination Well"] = dest_well
                        P200_components.at[i, "Transfer Volume [uL]"] = transf_vol
                    volume_left = volume_left - transf_vol
            
        i += 1

P20_components["Source Position"] = "P1" 
P20_components["Destination Position"] = "P2" 
P200_components["Source Position"] = "P1" 
P200_components["Destination Position"] = "P2" 

In [37]:
P20_components.head(5)

,Source Position,Source Well,Destination Position,Destination Well,Transfer Volume [uL]
0,P1,A1,P2,A1,10
1,P1,A1,P2,B1,10
2,P1,A1,P2,C1,10
3,P1,A1,P2,D1,10
4,P1,A1,P2,E1,10


## Calculate number of tips needed

In [38]:
num_transf_water_p200 = len(P200_water)
num_transf_water_p20 = len(P20_water)
num_transf_comp_p20 = len(P20_components)
num_transf_comp_p200 = len(P200_components)
num_transf_culture = len(P20_culture)
num_transf_kan_p20 = len(P20_kan)
num_transf_kan_p200 = len(P200_kan)
num_total_transf = num_transf_water_p200 + num_transf_water_p20 + num_transf_comp_p20 + num_transf_comp_p200+num_transf_kan_p20+num_transf_kan_p200+num_transf_culture
print(f'Number of transfers:')
print(f'\t Water (p200): {num_transf_water_p200}')
print(f'\t Water (p20): {num_transf_water_p20}')
print(f'\t Kan (p20): {num_transf_kan_p20}')
print(f'\t Kan (p200): {num_transf_kan_p200}')
print(f'\t Components (p200): {num_transf_comp_p200}')
print(f'\t Components (p20): {num_transf_comp_p20}')
print(f'\t Culture (p20): {num_transf_culture}')
print(f'Total number of transfers: {num_total_transf}')

Number of transfers:
	 Water (p200): 48
	 Water (p20): 0
	 Kan (p20): 48
	 Kan (p200): 0
	 Components (p200): 384
	 Components (p20): 336
	 Culture (p20): 48
Total number of transfers: 864


Calculate number of tip boxes needed:

In [39]:
print(f'This protocol requires:')
# For some reason, if transfering water (or from a reservoir plate) 
# biomek is using only 7 probes
num_tips_200 = 7 if num_transf_water_p200 > 0 else 0
num_tips_200 += num_transf_comp_p200 + num_transf_kan_p200
num_tips_20 = 7 if num_transf_water_p20 > 0 else 0
num_tips_20 += num_transf_comp_p20 + num_transf_kan_p20 + num_transf_culture
print(f'\t{np.ceil(num_tips_200 / 96):.0f} box(es) of p200 tips')
print(f'\t{np.ceil((num_tips_20) / 96):.0f} box(es) of p20 tips')


This protocol requires:
	5 box(es) of p200 tips
	5 box(es) of p20 tips


### Save biomek files

In [40]:
P200_water_file = f"{user_params['output_path']}/biomek_files/P200_water.csv"
P20_water_file = f"{user_params['output_path']}/biomek_files/P20_water.csv"
P20_kan_file = f"{user_params['output_path']}/biomek_files/P20_kan.csv"
P200_components_file = f"{user_params['output_path']}/biomek_files/P200_components.csv"
P20_components_file = f"{user_params['output_path']}/biomek_files/P20_components.csv"
P20_culture_file = f"{user_params['output_path']}/biomek_files/P20_culture.csv"

P200_water.to_csv(P200_water_file, index=False)
P20_water.to_csv(P20_water_file, index=False)
P20_kan.to_csv(P20_kan_file, index=False)
P200_components.to_csv(P200_components_file, index=False)
P20_components.to_csv(P20_components_file, index=False)
P20_culture.to_csv(P20_culture_file, index=False)